In [ ]:
!pip install --upgrade langchain langchain-core langchain-community langgraph transformers accelerate sentencepiece torch --quiet
!pip install huggingface-hub --quiet

import importlib
importlib.invalidate_caches()

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

MODEL = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

if torch.cuda.is_available():
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL, device_map="auto")
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256, do_sample=False)
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256, do_sample=False, device=-1)

hf_llm = HuggingFacePipeline(pipeline=pipe)

prompt = PromptTemplate.from_template("Conversation:\n{history}\nUser: {input}\nAssistant:")

chain = (
    {"history": RunnablePassthrough(), "input": RunnablePassthrough()}
    | prompt
    | hf_llm
    | StrOutputParser()
)

print(chain.invoke({"history": "", "input": "Explain overfitting in one sentence."}))

class State(TypedDict):
    input: str
    response: str

def llm_node(state: State):
    out = chain.invoke({"history": "", "input": state["input"]})
    return {"response": out}

def output_node(state: State):
    return {"response": state["response"]}

graph = StateGraph(State)
graph.add_node("llm", llm_node)
graph.add_node("out", output_node)
graph.add_edge(START, "llm")
graph.add_edge("llm", "out")
graph.add_edge("out", END)

app = graph.compile()

result = app.invoke({"input": "What is AI?"})
print(result["response"])


Device set to use cuda:0



AI is a graphical representation of a computer.
